# 文本分類實例

## Step1 導入相關包

In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset

## Step2 加載數據集

In [2]:
dataset = load_dataset("csv", data_files="./ChnSentiCorp_htl_all.csv", split="train")
dataset = dataset.filter(lambda x: x["review"] is not None)
dataset

Dataset({
    features: ['label', 'review'],
    num_rows: 7765
})

## Step3 劃分數據集

In [ ]:
datasets = dataset.train_test_split(test_size=0.1)
datasets

## Step4 數據集預處理

In [ ]:
import torch

tokenizer = AutoTokenizer.from_pretrained("hfl/rbt3")

def process_function(examples):
    tokenized_examples = tokenizer(examples["review"], max_length=128, truncation=True)
    tokenized_examples["labels"] = examples["label"]
    return tokenized_examples

tokenized_datasets = datasets.map(process_function, batched=True, remove_columns=datasets["train"].column_names)
tokenized_datasets

## Step5 創建模型

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained("hfl/rbt3")

In [ ]:
model.config

## Step6 創建評估函數

In [ ]:
import evaluate

acc_metric = evaluate.load("accuracy")
f1_metric = evaluate.load("f1")

In [ ]:
def eval_metric(eval_predict):
    predictions, labels = eval_predict
    predictions = predictions.argmax(axis=-1)
    acc = acc_metric.compute(predictions=predictions, references=labels)
    f1 = f1_metric.compute(predictions=predictions, references=labels)
    acc.update(f1)
    return acc

## Step7 創建TrainingArguments

In [ ]:
train_args = TrainingArguments(output_dir="./checkpoints",      # 輸出文件夾
                               per_device_train_batch_size=64,  # 訓練時的batch_size
                               per_device_eval_batch_size=128,  # 驗證時的batch_size
                               logging_steps=10,                # log 打印的頻率
                               evaluation_strategy="epoch",     # 評估策略
                               save_strategy="epoch",           # 保存策略
                               save_total_limit=3,              # 最大保存數
                               learning_rate=2e-5,              # 學習率
                               weight_decay=0.01,               # weight_decay
                               metric_for_best_model="f1",      # 設定評估指標
                               load_best_model_at_end=True)     # 訓練完成後加載最優模型
train_args

## Step8 創建Trainer

In [ ]:
from transformers import DataCollatorWithPadding
trainer = Trainer(model=model, 
                  args=train_args, 
                  train_dataset=tokenized_datasets["train"], 
                  eval_dataset=tokenized_datasets["test"], 
                  data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
                  compute_metrics=eval_metric)

## Step9 模型訓練

In [ ]:
trainer.train()

## Step10 模型評估

In [ ]:
trainer.evaluate(tokenized_datasets["test"])

## Step11 模型預測

In [ ]:
trainer.predict(tokenized_datasets["test"])

In [ ]:
from transformers import pipeline

id2_label = id2_label = {0: "差評！", 1: "好評！"}
model.config.id2label = id2_label
pipe = pipeline("text-classification", model=model, tokenizer=tokenizer, device=0)

In [ ]:
sen = "我覺得不錯！"
pipe(sen)